In [1]:
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutorRBD
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from pkg_resources import resource_filename
from simtk import unit
from perses.rjmc.topology_proposal import PointMutationEngineRBD

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [2]:
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
from simtk.openmm import app
from simtk import unit

import copy
import logging
import itertools
import os
import openeye.oechem as oechem
import numpy as np
import networkx as nx
import openmoltools.forcefield_generators as forcefield_generators
from perses.storage import NetCDFStorageView
from perses.rjmc.geometry import NoTorsionError
from perses.utils.rbd import edit_pdb_for_tleap, edit_tleap_in_inputs, edit_tleap_in_ions, generate_tleap_system
from functools import partial
from pkg_resources import resource_filename

try:
    from subprocess import getoutput  # If python 3
except ImportError:
    from commands import getoutput  # If python 2

class PointMutationEngineRBD2(PointMutationEngineRBD):
    def _generate_new_tleap_system(self, tleap_prefix, debug_dir, old_topology, new_topology, current_positions, is_complex):
        """
        Generates new system by: 1) mutating in pymol to get the new positions, 2) rearranging positions to match
        the atom order in the new_topology and copying solvent atoms from the old positions, 3) parametrizing the
        new system using tleap.
        
        Parameters
        ----------
        tleap_prefix : str
            Prefix for tleap input and output files
        debug_dir : str
            if specified, debug output files will be saved here
        old_topology : simtk.openmm.app.Topology object
            The old topology
        new_topology : simtk.openmm.app.Topology object
            The new topology
        current_positions : np.array
            The current positions
        is_complex : boolean
            Indicates whether the current system is apo or complex
        Returns
        -------
        new_positions : np.array
            The new positions
        new_system : simtk.openmm.System object
            The new system object        
        """
        _logger.info("Skipped PyMOL")
        name = 'rbd_ace2' if is_complex else 'rbd'
        
        # Prep PDBs for tleap
        _logger.info("Prepping PDBs for tleap")
        new_positions = self._prep_for_tleap(debug_dir, old_topology, new_topology, current_positions, is_complex)
        
        # Edit tleap in file
        tleap_prefix = os.path.join(debug_dir, f"5_{name}_mutant_tleap")
        mutant_template = resource_filename('perses', f'data/rbd-ace2/5_{name}_mutant_template_tleap.in')
        edit_tleap_in_inputs(mutant_template, tleap_prefix, debug_dir)

        # Generate system using tleap 
        _logger.info("Generating new system")
        _, _, new_system = generate_tleap_system(os.path.join(debug_dir, f"5_{name}_mutant_tleap"))
       
        return new_positions, new_system
    
    def _prep_for_tleap(self, debug_dir, old_topology, new_topology, current_positions, is_complex):
        """
        Given a mutated PDB, prepare a PDB for tleap input: 1) Rearrange the mutated PDB positions such that they 
        match the atom ordering in new_topology, 2) Copy the solvent positions from current_positions, 3) Save apo 
        RBD, apo ACE2 (for complex), solvent as separate PDBs.
        
        Parameters
        ----------
        debug_dir : str
            If specified, debug output files will be saved here
        old_topology : simtk.openmm.app.Topology object
            The old topology
        new_topology : simtk.openmm.app.Topology object
            The new topology
        current_positions : np.array
            The current positions
        is_complex : boolean
            Indicates whether the current system is apo or complex
        Returns
        -------
        new_positions : np.array
            The new positions
        """
        
        name = 'rbd_ace2' if is_complex else 'rbd'
        
        _logger.info("Skipped correcting mutated atom names")
        _logger.info(f"new topology atoms: {new_topology.getNumAtoms()}")
        
        # Load mutated (protonated) PDB
        if name == 'rbd':
            mutant_pdb = "/data/chodera/zhangi/perses_benchmark/neq/14/77/29_RBD_withh_aligned.pdb"
        else:
            mutant_pdb = "/data/chodera/zhangi/perses_benchmark/neq/14/77/29_RBD_ACE2_withh_aligned.pdb"
        mutated_pdb = app.PDBFile(mutant_pdb)
        mutated_n_atoms = mutated_pdb.topology.getNumAtoms()
        
        # Map atom indices from pymol PDB to atom indices in new_topology
        d_omm = {} # key: (atom name, residue id, chain id), value: atom index
        for atom_omm in new_topology.atoms():
            d_omm[(atom_omm.name, atom_omm.residue.id, atom_omm.residue.chain.id)] = atom_omm.index

        d_map = {} # key: atom index from pymol mutated PDB, value: atom index in new_topology
        for atom_pymol in mutated_pdb.topology.atoms():
            match_index = d_omm[(atom_pymol.name, atom_pymol.residue.id, atom_pymol.residue.chain.id)]
            d_map[atom_pymol.index] = match_index

        # Rearrange positions based on new_topology and add units to positions
        dim_1, dim_2 = np.array(mutated_pdb.positions).shape
        mutated_positions = unit.Quantity(np.zeros(shape=(dim_1, dim_2)), unit=unit.nanometers)
        positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in mutated_pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
        for k, v in d_map.items():
            mutated_positions[v] = positions[k]
        
        # Copy solvent positions from old positions
        solvent_atoms = [atom for atom in old_topology.atoms() if atom.residue.chain.id == 'Y']
        first_solvent_atom = solvent_atoms[0].index
        new_positions = unit.Quantity(np.zeros([mutated_n_atoms + len(solvent_atoms), 3]), unit=unit.nanometers)
        new_positions[:mutated_n_atoms, :] = mutated_positions
        new_positions[mutated_n_atoms:, :] = current_positions[first_solvent_atom:]
        _logger.info(f"new positions atoms: {new_positions.shape}")
        def save_apo(output_pdb, topology, positions, chains_to_keep):
            modeller = app.Modeller(topology, positions)
            to_delete = []
            for chain in modeller.topology.chains():
                if chain.id not in chains_to_keep:
                    to_delete.append(chain)
            modeller.delete(to_delete)
            app.PDBFile.writeFile(modeller.topology, modeller.positions, open(output_pdb, "w"), keepIds=True)

        # Save apo solute PDBs and then correct for tleap
        rbd_pdb = os.path.join(debug_dir, f"3_{name}_mutant_rbd_tleap.pdb")
        rbd_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_rbd_tleap_final.pdb')
        save_apo(rbd_pdb, new_topology, new_positions, ['R', 'X'])
        edit_pdb_for_tleap(rbd_pdb, rbd_pdb_final)
        
        solvent_pdb = os.path.join(debug_dir, f"3_{name}_mutant_solvent_tleap.pdb")
        solvent_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_solvent_tleap_final.pdb')
        save_apo(solvent_pdb, new_topology, new_positions, ['Y'])
        edit_pdb_for_tleap(solvent_pdb, solvent_pdb_final)
        
        if is_complex:
            ace2_pdb = os.path.join(debug_dir, f"3_{name}_mutant_ace2_tleap.pdb")
            ace2_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_ace2_tleap_final.pdb')
            save_apo(ace2_pdb, new_topology, new_positions, ['C', 'D', 'E'])
            edit_pdb_for_tleap(ace2_pdb, ace2_pdb_final, is_ace2=True)

        return new_positions

In [24]:
from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol, oechem
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine, PointMutationEngineRBD
from perses.rjmc.geometry import FFAllAngleGeometryEngine
from perses.utils.rbd import edit_pdb_for_tleap, edit_tleap_in_inputs, edit_tleap_in_ions, generate_tleap_system

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator
import os
from pkg_resources import resource_filename
import shutil
import tempfile

ENERGY_THRESHOLD = 1e-2
temperature = 298 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']
CL_CHARGE = unit.Quantity(value=-1.0, unit=unit.elementary_charge)
CL_SIGMA = unit.Quantity(value=0.4477656957373345, unit=unit.nanometer)
CL_EPSILON = unit.Quantity(value=0.14891274399999999, unit=unit.kilojoule_per_mole)
NA_CHARGE = unit.Quantity(value=1.0, unit=unit.elementary_charge)
NA_SIGMA = unit.Quantity(value=0.2439280690268249, unit=unit.nanometer)
NA_EPSILON = unit.Quantity(value=0.3658460312, unit=unit.kilojoule_per_mole)
O_CHARGE = unit.Quantity(value=-0.834, unit=unit.elementary_charge)
H_CHARGE = unit.Quantity(value=0.417, unit=unit.elementary_charge)


class PointMutationExecutorRBD2(PointMutationExecutorRBD):
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 clean=False,
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 vanilla=True,
                 repartitioned=True,
                 debug_dir=None,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            clean : bool, default False
                whether to clean the PDB for tleap
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            vanilla : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            repartitioned : bool, default True
                whether to generate a RepartitionedHybridTopologyFactory
            debug_dir : str, default None
                if specified, debug output files will be saved here
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        # Make debug directory
        is_temp = False
        if debug_dir:
            if not os.path.exists(debug_dir):
                os.system(f"mkdir {debug_dir}")
        else:
            debug_dir = tempfile.mkdtemp()
            is_temp = True        

        ## Generate the old topology, positions, and system
        # Prep PDBs for tleap
        _logger.info("Editing PDBs for tleap")
        protein_name = os.path.basename(protein_filename)
        ligand_name = os.path.basename(ligand_input)
        protein_tleap = os.path.join(debug_dir, f"{protein_name[:-4]}_tleap.pdb")
        ligand_tleap = os.path.join(debug_dir, f"{ligand_name[:-4]}_tleap.pdb")
        if clean:
            edit_pdb_for_tleap(protein_filename, protein_tleap)
            edit_pdb_for_tleap(ligand_input, ligand_tleap)
        else:
            os.system(f"cp {protein_filename} {protein_tleap}")
            os.system(f"cp {ligand_input} {ligand_tleap}")
        
        # Edit tleap files
        _logger.info("Editing tleap.in input files")
        apo_tleap_prefix = os.path.join(debug_dir, "1_rbd_tleap")
        complex_tleap_prefix = os.path.join(debug_dir, "1_rbd_ace2_tleap")
        apo_template = resource_filename('perses', 'data/rbd-ace2/1_rbd_template_tleap.in')
        complex_template = resource_filename('perses', 'data/rbd-ace2/1_rbd_ace2_template_tleap.in')
        edit_tleap_in_inputs(apo_template, apo_tleap_prefix, debug_dir)
        edit_tleap_in_inputs(complex_template, complex_tleap_prefix, debug_dir)
 
        _logger.info("Editing tleap.in number of ions")
        edit_tleap_in_ions(apo_tleap_prefix)
        edit_tleap_in_ions(complex_tleap_prefix)
        
        # Generating old systems
        _logger.info("Generating solvated old systems")
        apo_topology, apo_positions, apo_system = generate_tleap_system(apo_tleap_prefix)
        complex_topology, complex_positions, complex_system = generate_tleap_system(complex_tleap_prefix)
        
        # Correct the topologies
        _logger.info("Correcting tleap topologies")
        apo_topology_corrected = self._correct_topology(apo_topology)
        complex_topology_corrected = self._correct_topology(complex_topology, is_apo=False)
        
        # Format inputs for pipeline
        inputs = [[apo_topology_corrected, apo_positions, apo_system, apo_tleap_prefix, False], [complex_topology_corrected, complex_positions, complex_system, complex_tleap_prefix, True]]
    
        # Make system generator -- note this is only for system_generator.forcefield call in PointMutationEngine init
        molecules = []
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)
        
        # Run pipeline...
        htfs = []
        for (top, pos, sys, tleap_prefix, is_complex) in inputs:
            name = 'rbd_ace2' if is_complex else 'rbd'
            _logger.info(f"Generating topology proposal for {name}")
            point_mutation_engine = PointMutationEngineRBD2(wildtype_topology=top,
                                                         system_generator=self.system_generator,
                                                         chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                         max_point_mutants=1,
                                                         residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                         allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                         aggregate=True) # Always allow aggregation

            topology_proposal, new_positions = point_mutation_engine.propose(sys, top, pos, tleap_prefix, is_complex, debug_dir)

            # Fix naked charges in old and new systems
            for system in [topology_proposal.old_system, topology_proposal.new_system]:
                force_dict = {i.__class__.__name__: i for i in system.getForces()}
                if 'NonbondedForce' in [i for i in force_dict.keys()]:
                    nb_force = force_dict['NonbondedForce']
                    for i in range(nb_force.getNumParticles()):
                        charge, sigma, epsilon = nb_force.getParticleParameters(i)
                        if sigma == 0*unit.nanometer:
                            sigma = 0.6*unit.nanometer
                            nb_force.setParticleParameters(i, charge, sigma, epsilon)
                        if epsilon == 0*unit.kilojoule_per_mole:
                            epsilon = 0.01*unit.kilojoule_per_mole
                            nb_force.setParticleParameters(i, charge, sigma, epsilon)
            
            # Check for charge change...
            charge_diff = point_mutation_engine._get_charge_difference(current_resname = topology_proposal._old_topology.residue_topology.name,
                                                                       new_resname = topology_proposal._new_topology.residue_topology.name)
            _logger.info(f"charge diff: {charge_diff}")
            if charge_diff != 0:
                new_water_indices_to_ionize = point_mutation_engine.get_water_indices(charge_diff, new_positions, topology_proposal._new_topology, radius=0.8)
                _logger.info(f"new water indices to ionize {new_water_indices_to_ionize}")
                PointMutationExecutorRBD._modify_new_system(new_water_indices_to_ionize, topology_proposal._new_system, charge_diff)
                PointMutationExecutorRBD._modify_atom_classes(new_water_indices_to_ionize, topology_proposal)

            factories = []
            if vanilla:
                repartitioned_endstate = None
                self.generate_htf(HybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)
            if repartitioned:
                for repartitioned_endstate in [0, 1]:
                    self.generate_htf(RepartitionedHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)


In [25]:
# Read args
# outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/77/"
outdir = "/data/chodera/zhangi/perses_benchmark/repex/31/29/0"
residue = '501'
mutant = "TYR"

rbd_file = resource_filename('perses', 'data/rbd-ace2/0_rbd.pdb') 
ace2_file = resource_filename('perses', 'data/rbd-ace2/0_ace2.pdb')


In [26]:
solvent_delivery = PointMutationExecutorRBD2(rbd_file,
                        'R',
                        residue,
                        mutant,
                        ligand_input=ace2_file,
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        debug_dir=os.path.join(outdir, "debug/")
                       )



INFO:root:Editing PDBs for tleap
INFO:root:Editing tleap.in input files
INFO:root:Editing tleap.in number of ions


num positive: 38
num negative: 40.0


INFO:root:Generating solvated old systems


num positive: 172.0
num negative: 152


INFO:root:Added barostat!
INFO:root:Added barostat!
INFO:root:Correcting tleap topologies
INFO:root:Generating topology proposal for rbd
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Adding new atoms
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{4: 4, 14: 10, 15: 11}]
INFO:proposal_generator:{4: 4, 14: 10, 15: 11}
INFO:proposal_generator:0
INFO:proposal_generator:Checking maps to see if they break rings
INFO:proposal_generator:Only one map so returning that one
INFO:proposal_generator:{4: 4, 14: 10, 15: 11}
INFO:root:Skipped PyMOL
INFO:root:Prepping PDBs for tleap
INFO:root:Skipped correcting mutated atom names
INFO:root:new topology atoms: 44750
INFO:root:new positions atoms: (44750, 3)
INFO:root:Generating new system
INFO:root:Added barostat!
INFO:proposal_generator:Finishing up topology proposal
INFO:root:charge diff: 0
INFO:rela

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Adding new atoms
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{4: 4, 14: 10, 15: 11}]
INFO:proposal_generator:{4: 4, 14: 10, 15: 11}
INFO:proposal_generator:0
INFO:proposal_generator:Checking maps to see if they break rings
INFO:proposal_generator:Only one map so returning that one
INFO:proposal_generator:{4: 4, 14: 10, 15: 11}
INFO:root:Skipped PyMOL
INFO:root:Prepping PDBs for tleap
INFO:root:Skipped correcting mutated atom names
INFO:root:new topology atoms: 183585
INFO:root:new positions atoms: (183585, 3)
INFO:root:Generating new system
INFO:root:Added barostat!
INFO:proposal_generator:Finishing up topology proposal
INFO:root:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exception

In [27]:
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [7]:
i = os.path.basename(os.path.dirname(outdir))

with open(os.path.join(outdir, f"{i}_apo.pickle"), "rb") as f:
    htf = pickle.load(f)


In [8]:
htf._topology_proposal.new_system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2ab37ad41de0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2ab37ad41b10> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2ab37ad41c90> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ab37ad417e0> >,
 <simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ab37ad41e70> >]

In [10]:
nb_force = htf._topology_proposal.new_system.getForce(3)

In [ ]:
unit.kilojoule_per_mole

In [22]:
for i in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(i)
    if sigma == 0*unit.nanometer:
        sigma = 0.6*unit.nanometer
        nb_force.setParticleParameters(i, charge, sigma, epsilon)
    if epsilon == 0*unit.kilojoule_per_mole:
        epsilon = 0.01*unit.kilojoule_per_mole
        nb_force.setParticleParameters(i, charge, sigma, epsilon)
#     if sigma == 0 * unit.nanometer:
#         print(i, charge, sigma, epsilon)

In [16]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom.index, atom)

0 <Atom 0 (H1) of chain 0 residue 0 (ACE)>
1 <Atom 1 (CH3) of chain 0 residue 0 (ACE)>
2 <Atom 2 (H2) of chain 0 residue 0 (ACE)>
3 <Atom 3 (H3) of chain 0 residue 0 (ACE)>
4 <Atom 4 (C) of chain 0 residue 0 (ACE)>
5 <Atom 5 (O) of chain 0 residue 0 (ACE)>
6 <Atom 6 (N) of chain 0 residue 1 (THR)>
7 <Atom 7 (H) of chain 0 residue 1 (THR)>
8 <Atom 8 (CA) of chain 0 residue 1 (THR)>
9 <Atom 9 (HA) of chain 0 residue 1 (THR)>
10 <Atom 10 (CB) of chain 0 residue 1 (THR)>
11 <Atom 11 (HB) of chain 0 residue 1 (THR)>
12 <Atom 12 (CG2) of chain 0 residue 1 (THR)>
13 <Atom 13 (HG21) of chain 0 residue 1 (THR)>
14 <Atom 14 (HG22) of chain 0 residue 1 (THR)>
15 <Atom 15 (HG23) of chain 0 residue 1 (THR)>
16 <Atom 16 (OG1) of chain 0 residue 1 (THR)>
17 <Atom 17 (HG1) of chain 0 residue 1 (THR)>
18 <Atom 18 (C) of chain 0 residue 1 (THR)>
19 <Atom 19 (O) of chain 0 residue 1 (THR)>
20 <Atom 20 (N) of chain 0 residue 2 (ASN)>
21 <Atom 21 (H) of chain 0 residue 2 (ASN)>
22 <Atom 22 (CA) of chain 0 

1447 <Atom 1447 (HD23) of chain 0 residue 93 (LEU)>
1448 <Atom 1448 (C) of chain 0 residue 93 (LEU)>
1449 <Atom 1449 (O) of chain 0 residue 93 (LEU)>
1450 <Atom 1450 (N) of chain 0 residue 94 (PRO)>
1451 <Atom 1451 (CD) of chain 0 residue 94 (PRO)>
1452 <Atom 1452 (HD2) of chain 0 residue 94 (PRO)>
1453 <Atom 1453 (HD3) of chain 0 residue 94 (PRO)>
1454 <Atom 1454 (CG) of chain 0 residue 94 (PRO)>
1455 <Atom 1455 (HG2) of chain 0 residue 94 (PRO)>
1456 <Atom 1456 (HG3) of chain 0 residue 94 (PRO)>
1457 <Atom 1457 (CB) of chain 0 residue 94 (PRO)>
1458 <Atom 1458 (HB2) of chain 0 residue 94 (PRO)>
1459 <Atom 1459 (HB3) of chain 0 residue 94 (PRO)>
1460 <Atom 1460 (CA) of chain 0 residue 94 (PRO)>
1461 <Atom 1461 (HA) of chain 0 residue 94 (PRO)>
1462 <Atom 1462 (C) of chain 0 residue 94 (PRO)>
1463 <Atom 1463 (O) of chain 0 residue 94 (PRO)>
1464 <Atom 1464 (N) of chain 0 residue 95 (ASP)>
1465 <Atom 1465 (H) of chain 0 residue 95 (ASP)>
1466 <Atom 1466 (CA) of chain 0 residue 95 (ASP)>

2697 <Atom 2697 (HD3) of chain 0 residue 175 (PRO)>
2698 <Atom 2698 (CG) of chain 0 residue 175 (PRO)>
2699 <Atom 2699 (HG2) of chain 0 residue 175 (PRO)>
2700 <Atom 2700 (HG3) of chain 0 residue 175 (PRO)>
2701 <Atom 2701 (CB) of chain 0 residue 175 (PRO)>
2702 <Atom 2702 (HB2) of chain 0 residue 175 (PRO)>
2703 <Atom 2703 (HB3) of chain 0 residue 175 (PRO)>
2704 <Atom 2704 (CA) of chain 0 residue 175 (PRO)>
2705 <Atom 2705 (HA) of chain 0 residue 175 (PRO)>
2706 <Atom 2706 (C) of chain 0 residue 175 (PRO)>
2707 <Atom 2707 (O) of chain 0 residue 175 (PRO)>
2708 <Atom 2708 (N) of chain 0 residue 176 (TYR)>
2709 <Atom 2709 (H) of chain 0 residue 176 (TYR)>
2710 <Atom 2710 (CA) of chain 0 residue 176 (TYR)>
2711 <Atom 2711 (HA) of chain 0 residue 176 (TYR)>
2712 <Atom 2712 (CB) of chain 0 residue 176 (TYR)>
2713 <Atom 2713 (HB2) of chain 0 residue 176 (TYR)>
2714 <Atom 2714 (HB3) of chain 0 residue 176 (TYR)>
2715 <Atom 2715 (CG) of chain 0 residue 176 (TYR)>
2716 <Atom 2716 (CD1) of cha

4197 <Atom 4197 (H1) of chain 2 residue 592 (HOH)>
4198 <Atom 4198 (H2) of chain 2 residue 592 (HOH)>
4199 <Atom 4199 (O) of chain 2 residue 593 (HOH)>
4200 <Atom 4200 (H1) of chain 2 residue 593 (HOH)>
4201 <Atom 4201 (H2) of chain 2 residue 593 (HOH)>
4202 <Atom 4202 (O) of chain 2 residue 594 (HOH)>
4203 <Atom 4203 (H1) of chain 2 residue 594 (HOH)>
4204 <Atom 4204 (H2) of chain 2 residue 594 (HOH)>
4205 <Atom 4205 (O) of chain 2 residue 595 (HOH)>
4206 <Atom 4206 (H1) of chain 2 residue 595 (HOH)>
4207 <Atom 4207 (H2) of chain 2 residue 595 (HOH)>
4208 <Atom 4208 (O) of chain 2 residue 596 (HOH)>
4209 <Atom 4209 (H1) of chain 2 residue 596 (HOH)>
4210 <Atom 4210 (H2) of chain 2 residue 596 (HOH)>
4211 <Atom 4211 (O) of chain 2 residue 597 (HOH)>
4212 <Atom 4212 (H1) of chain 2 residue 597 (HOH)>
4213 <Atom 4213 (H2) of chain 2 residue 597 (HOH)>
4214 <Atom 4214 (O) of chain 2 residue 598 (HOH)>
4215 <Atom 4215 (H1) of chain 2 residue 598 (HOH)>
4216 <Atom 4216 (H2) of chain 2 resid

5697 <Atom 5697 (H1) of chain 2 residue 1092 (HOH)>
5698 <Atom 5698 (H2) of chain 2 residue 1092 (HOH)>
5699 <Atom 5699 (O) of chain 2 residue 1093 (HOH)>
5700 <Atom 5700 (H1) of chain 2 residue 1093 (HOH)>
5701 <Atom 5701 (H2) of chain 2 residue 1093 (HOH)>
5702 <Atom 5702 (O) of chain 2 residue 1094 (HOH)>
5703 <Atom 5703 (H1) of chain 2 residue 1094 (HOH)>
5704 <Atom 5704 (H2) of chain 2 residue 1094 (HOH)>
5705 <Atom 5705 (O) of chain 2 residue 1095 (HOH)>
5706 <Atom 5706 (H1) of chain 2 residue 1095 (HOH)>
5707 <Atom 5707 (H2) of chain 2 residue 1095 (HOH)>
5708 <Atom 5708 (O) of chain 2 residue 1096 (HOH)>
5709 <Atom 5709 (H1) of chain 2 residue 1096 (HOH)>
5710 <Atom 5710 (H2) of chain 2 residue 1096 (HOH)>
5711 <Atom 5711 (O) of chain 2 residue 1097 (HOH)>
5712 <Atom 5712 (H1) of chain 2 residue 1097 (HOH)>
5713 <Atom 5713 (H2) of chain 2 residue 1097 (HOH)>
5714 <Atom 5714 (O) of chain 2 residue 1098 (HOH)>
5715 <Atom 5715 (H1) of chain 2 residue 1098 (HOH)>
5716 <Atom 5716 (H

...

38441 <Atom 38441 (O) of chain 2 residue 12007 (HOH)>
38442 <Atom 38442 (H1) of chain 2 residue 12007 (HOH)>
38443 <Atom 38443 (H2) of chain 2 residue 12007 (HOH)>
38444 <Atom 38444 (O) of chain 2 residue 12008 (HOH)>
38445 <Atom 38445 (H1) of chain 2 residue 12008 (HOH)>
38446 <Atom 38446 (H2) of chain 2 residue 12008 (HOH)>
38447 <Atom 38447 (O) of chain 2 residue 12009 (HOH)>
38448 <Atom 38448 (H1) of chain 2 residue 12009 (HOH)>
38449 <Atom 38449 (H2) of chain 2 residue 12009 (HOH)>
38450 <Atom 38450 (O) of chain 2 residue 12010 (HOH)>
38451 <Atom 38451 (H1) of chain 2 residue 12010 (HOH)>
38452 <Atom 38452 (H2) of chain 2 residue 12010 (HOH)>
38453 <Atom 38453 (O) of chain 2 residue 12011 (HOH)>
38454 <Atom 38454 (H1) of chain 2 residue 12011 (HOH)>
38455 <Atom 38455 (H2) of chain 2 residue 12011 (HOH)>
38456 <Atom 38456 (O) of chain 2 residue 12012 (HOH)>
38457 <Atom 38457 (H1) of chain 2 residue 12012 (HOH)>
38458 <Atom 38458 (H2) of chain 2 residue 12012 (HOH)>
38459 <Atom 3845

39941 <Atom 39941 (O) of chain 2 residue 12507 (HOH)>
39942 <Atom 39942 (H1) of chain 2 residue 12507 (HOH)>
39943 <Atom 39943 (H2) of chain 2 residue 12507 (HOH)>
39944 <Atom 39944 (O) of chain 2 residue 12508 (HOH)>
39945 <Atom 39945 (H1) of chain 2 residue 12508 (HOH)>
39946 <Atom 39946 (H2) of chain 2 residue 12508 (HOH)>
39947 <Atom 39947 (O) of chain 2 residue 12509 (HOH)>
39948 <Atom 39948 (H1) of chain 2 residue 12509 (HOH)>
39949 <Atom 39949 (H2) of chain 2 residue 12509 (HOH)>
39950 <Atom 39950 (O) of chain 2 residue 12510 (HOH)>
39951 <Atom 39951 (H1) of chain 2 residue 12510 (HOH)>
39952 <Atom 39952 (H2) of chain 2 residue 12510 (HOH)>
39953 <Atom 39953 (O) of chain 2 residue 12511 (HOH)>
39954 <Atom 39954 (H1) of chain 2 residue 12511 (HOH)>
39955 <Atom 39955 (H2) of chain 2 residue 12511 (HOH)>
39956 <Atom 39956 (O) of chain 2 residue 12512 (HOH)>
39957 <Atom 39957 (H1) of chain 2 residue 12512 (HOH)>
39958 <Atom 39958 (H2) of chain 2 residue 12512 (HOH)>
39959 <Atom 3995

41441 <Atom 41441 (O) of chain 2 residue 13007 (HOH)>
41442 <Atom 41442 (H1) of chain 2 residue 13007 (HOH)>
41443 <Atom 41443 (H2) of chain 2 residue 13007 (HOH)>
41444 <Atom 41444 (O) of chain 2 residue 13008 (HOH)>
41445 <Atom 41445 (H1) of chain 2 residue 13008 (HOH)>
41446 <Atom 41446 (H2) of chain 2 residue 13008 (HOH)>
41447 <Atom 41447 (O) of chain 2 residue 13009 (HOH)>
41448 <Atom 41448 (H1) of chain 2 residue 13009 (HOH)>
41449 <Atom 41449 (H2) of chain 2 residue 13009 (HOH)>
41450 <Atom 41450 (O) of chain 2 residue 13010 (HOH)>
41451 <Atom 41451 (H1) of chain 2 residue 13010 (HOH)>
41452 <Atom 41452 (H2) of chain 2 residue 13010 (HOH)>
41453 <Atom 41453 (O) of chain 2 residue 13011 (HOH)>
41454 <Atom 41454 (H1) of chain 2 residue 13011 (HOH)>
41455 <Atom 41455 (H2) of chain 2 residue 13011 (HOH)>
41456 <Atom 41456 (O) of chain 2 residue 13012 (HOH)>
41457 <Atom 41457 (H1) of chain 2 residue 13012 (HOH)>
41458 <Atom 41458 (H2) of chain 2 residue 13012 (HOH)>
41459 <Atom 4145

42940 <Atom 42940 (H2) of chain 2 residue 13506 (HOH)>
42941 <Atom 42941 (O) of chain 2 residue 13507 (HOH)>
42942 <Atom 42942 (H1) of chain 2 residue 13507 (HOH)>
42943 <Atom 42943 (H2) of chain 2 residue 13507 (HOH)>
42944 <Atom 42944 (O) of chain 2 residue 13508 (HOH)>
42945 <Atom 42945 (H1) of chain 2 residue 13508 (HOH)>
42946 <Atom 42946 (H2) of chain 2 residue 13508 (HOH)>
42947 <Atom 42947 (O) of chain 2 residue 13509 (HOH)>
42948 <Atom 42948 (H1) of chain 2 residue 13509 (HOH)>
42949 <Atom 42949 (H2) of chain 2 residue 13509 (HOH)>
42950 <Atom 42950 (O) of chain 2 residue 13510 (HOH)>
42951 <Atom 42951 (H1) of chain 2 residue 13510 (HOH)>
42952 <Atom 42952 (H2) of chain 2 residue 13510 (HOH)>
42953 <Atom 42953 (O) of chain 2 residue 13511 (HOH)>
42954 <Atom 42954 (H1) of chain 2 residue 13511 (HOH)>
42955 <Atom 42955 (H2) of chain 2 residue 13511 (HOH)>
42956 <Atom 42956 (O) of chain 2 residue 13512 (HOH)>
42957 <Atom 42957 (H1) of chain 2 residue 13512 (HOH)>
42958 <Atom 4295

44440 <Atom 44440 (H2) of chain 2 residue 14006 (HOH)>
44441 <Atom 44441 (O) of chain 2 residue 14007 (HOH)>
44442 <Atom 44442 (H1) of chain 2 residue 14007 (HOH)>
44443 <Atom 44443 (H2) of chain 2 residue 14007 (HOH)>
44444 <Atom 44444 (O) of chain 2 residue 14008 (HOH)>
44445 <Atom 44445 (H1) of chain 2 residue 14008 (HOH)>
44446 <Atom 44446 (H2) of chain 2 residue 14008 (HOH)>
44447 <Atom 44447 (O) of chain 2 residue 14009 (HOH)>
44448 <Atom 44448 (H1) of chain 2 residue 14009 (HOH)>
44449 <Atom 44449 (H2) of chain 2 residue 14009 (HOH)>
44450 <Atom 44450 (O) of chain 2 residue 14010 (HOH)>
44451 <Atom 44451 (H1) of chain 2 residue 14010 (HOH)>
44452 <Atom 44452 (H2) of chain 2 residue 14010 (HOH)>
44453 <Atom 44453 (O) of chain 2 residue 14011 (HOH)>
44454 <Atom 44454 (H1) of chain 2 residue 14011 (HOH)>
44455 <Atom 44455 (H2) of chain 2 residue 14011 (HOH)>
44456 <Atom 44456 (O) of chain 2 residue 14012 (HOH)>
44457 <Atom 44457 (H1) of chain 2 residue 14012 (HOH)>
44458 <Atom 4445